# Accessing strain survey data using MPD

In this example we will programmatically retrieve a measure record from MPD. The measure we are working with here reflects total locomotor activity 3h post cocaine injection over a test period of 4 hours. This measure has an MPD measure ID of 40708 and is named activ_cocaine2_3. A total of 15 strains were assayed across both sexs with an average n of 8 mice per sex per strain. 

We will first retrieve individual animal values for this measure.

In [10]:
import requests
import json
from pandas.io.json import json_normalize

resp = requests.get("https://phenome.jax.org/api/pheno/animalvals/40708") 
data = json.loads(resp.text)
animaldata = json_normalize(data['animaldata'])
print (animaldata)

#fig = interaction_plot(animaldata.sex, animaldata.strain, animaldata.value)


#with open("queryMPD_SingleMeasure.tsv", 'w') as fl:    
#    fl.writelines(resp.text)

NameError: name 'log' is not defined

# Assess strain x sex effects

Now let us fit a linear model that assesses the main effects of strain and sex in addition to strain by sex effects


In [2]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot

formula = 'value ~ C(sex) + C(strain) + C(strain):C(sex)'
model = ols(formula, animaldata).fit()
aov_table = anova_lm(model)

print(aov_table)


                     df        sum_sq       mean_sq          F        PR(>F)
C(sex)              1.0  2.931693e+07  2.931693e+07   8.435830  4.088519e-03
C(strain)          14.0  1.172687e+09  8.376333e+07  24.102563  9.216382e-36
C(strain):C(sex)   14.0  1.231190e+08  8.794214e+06   2.530500  2.365076e-03
Residual          202.0  7.020080e+08  3.475287e+06        NaN           NaN


# Estimating heritability

Quite simply, heritability is defined as the ratio of genotypic and phenotypic variance. Therefore to estimate heritability we will need to obtain the between subject variance (genetic variance) along with the within subject variance (error variance) and interaction variance contained in the strain:sex term.

In [3]:
msb = aov_table.mean_sq[1] #between subject variance
mse = aov_table.mean_sq[3] #within subject variance
msi = aov_table.mean_sq[2] #interaction variance
s = 2
n = 8

h2 = round((msb-msi)/(s*(msb+(n-1)*mse)),2)

print("heritability for this measure is : ", h2)


heritability for this measure is :  0.35
